In [2]:
from neo4j import GraphDatabase as Graph

In [6]:
class HelloWorldExample(object):

    def __init__(self, uri, user, password):
        self._driver = Graph.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()

    def print_greeting(self, message):
        with self._driver.session() as session:
            greeting = session.write_transaction(self._create_and_return_greeting, message)
            print(greeting)

    @staticmethod
    def _create_and_return_greeting(tx, message):
        result = tx.run("CREATE (a:Greeting) "
                        "SET a.message = $message "
                        "RETURN a.message + ', from node ' + id(a)", message=message)
        return result.single()[0]

In [8]:
HelloWorldExample('bolt://localhost:11002','neo4j','12345')

In [45]:
from py2neo import Graph, Node, Relationship

In [36]:
url="bolt://localhost:11002"
user="neo4j"
pwd="12345"

In [37]:
KG = Graph(url,auth=(user, pwd))

In [53]:
tx = KG.begin()
a = Node("Human", name="Alice")
tx.create(a) 
b = Node("Human", name="Bob")
tx.create(b)
ab = Relationship(a,"KNOWS",b)
tx.create(ab)
tx.commit()

In [55]:
KG.run("MATCH (a:Person) RETURN a.name, a.born LIMIT 20").data()

[{'a.born': 1964, 'a.name': 'Keanu Reeves'},
 {'a.born': 1967, 'a.name': 'Carrie-Anne Moss'},
 {'a.born': 1961, 'a.name': 'Laurence Fishburne'},
 {'a.born': 1960, 'a.name': 'Hugo Weaving'},
 {'a.born': 1967, 'a.name': 'Lilly Wachowski'},
 {'a.born': 1965, 'a.name': 'Lana Wachowski'},
 {'a.born': 1952, 'a.name': 'Joel Silver'},
 {'a.born': 1978, 'a.name': 'Emil Eifrem'},
 {'a.born': 1975, 'a.name': 'Charlize Theron'},
 {'a.born': 1940, 'a.name': 'Al Pacino'},
 {'a.born': 1944, 'a.name': 'Taylor Hackford'},
 {'a.born': 1962, 'a.name': 'Tom Cruise'},
 {'a.born': 1937, 'a.name': 'Jack Nicholson'},
 {'a.born': 1962, 'a.name': 'Demi Moore'},
 {'a.born': 1958, 'a.name': 'Kevin Bacon'},
 {'a.born': 1966, 'a.name': 'Kiefer Sutherland'},
 {'a.born': 1971, 'a.name': 'Noah Wyle'},
 {'a.born': 1968, 'a.name': 'Cuba Gooding Jr.'},
 {'a.born': 1957, 'a.name': 'Kevin Pollak'},
 {'a.born': 1943, 'a.name': 'J.T. Walsh'}]

In [56]:
KG.run("MATCH (p:Person)-[:WROTE]->(:Movie {title: 'Speed Racer'}) RETURN p.name").data()

[{'p.name': 'Lana Wachowski'}, {'p.name': 'Lilly Wachowski'}]

In [72]:
Hi = KG.run("MATCH (m:Movie)<--(:Person {name: 'Tom Hanks'}) RETURN m.title")

In [73]:
Hi

In [74]:
Hi.data()

[{'m.title': 'Apollo 13'},
 {'m.title': "You've Got Mail"},
 {'m.title': 'A League of Their Own'},
 {'m.title': 'Joe Versus the Volcano'},
 {'m.title': 'That Thing You Do'},
 {'m.title': 'The Da Vinci Code'},
 {'m.title': 'Cloud Atlas'},
 {'m.title': 'Cast Away'},
 {'m.title': 'The Green Mile'},
 {'m.title': 'Sleepless in Seattle'},
 {'m.title': 'The Polar Express'},
 {'m.title': "Charlie Wilson's War"},
 {'m.title': 'That Thing You Do'}]

In [86]:
import pandas as pandas
pandas.DataFrame(KG.run("MATCH (m:Movie)-[rel]-(p:Person {name: 'Tom Hanks'}) RETURN m.title AS movieName,type(rel) AS relationType, p.name AS actorName").data())

,actorName,movieName,relationType
0,Tom Hanks,Apollo 13,ACTED_IN
1,Tom Hanks,You've Got Mail,ACTED_IN
2,Tom Hanks,A League of Their Own,ACTED_IN
3,Tom Hanks,Joe Versus the Volcano,ACTED_IN
4,Tom Hanks,That Thing You Do,ACTED_IN
5,Tom Hanks,The Da Vinci Code,ACTED_IN
6,Tom Hanks,Cloud Atlas,ACTED_IN
7,Tom Hanks,Cast Away,ACTED_IN
8,Tom Hanks,The Green Mile,ACTED_IN
9,Tom Hanks,Sleepless in Seattle,ACTED_IN


In [84]:
pandas.DataFrame(KG.run("MATCH (m:Movie)-[rel:ACTED_IN]-(:Person {name: 'Tom Hanks'}) RETURN m.title, rel.roles").data())

,m.title,rel.roles
0,Apollo 13,[Jim Lovell]
1,You've Got Mail,[Joe Fox]
2,A League of Their Own,[Jimmy Dugan]
3,Joe Versus the Volcano,[Joe Banks]
4,That Thing You Do,[Mr. White]
5,The Da Vinci Code,[Dr. Robert Langdon]
6,Cloud Atlas,"[Zachry, Dr. Henry Goose, Isaac Sachs, Dermot ..."
7,Cast Away,[Chuck Noland]
8,The Green Mile,[Paul Edgecomb]
9,Sleepless in Seattle,[Sam Baldwin]


In [87]:
pandas.DataFrame(KG.run("MATCH (a:Person)-[:ACTED_IN]->(m:Movie) WHERE a.name = 'Tom Cruise' RETURN m.title as Movie").data())

,Movie
0,Jerry Maguire
1,Top Gun
2,A Few Good Men


In [88]:
Query = "MATCH (a:Person)-[:ACTED_IN]->(m:Movie) WHERE a.born > 1960 AND m.title = 'The Matrix' RETURN a.name as Name, a.born as `Year Born`"

In [90]:
pandas.DataFrame(KG.run(Query).data())

,Name,Year Born
0,Emil Eifrem,1978
1,Laurence Fishburne,1961
2,Carrie-Anne Moss,1967
3,Keanu Reeves,1964
